In [2]:
# source/s: 
# set-up gspread
# https://www.youtube.com/watch?v=bu5wXjz2KvU
#
# https://console.cloud.google.com

# pip install pydrive
# pip instll gspread
# pip install pandas

import pandas as pd
import numpy as np
import gspread

In [3]:
# after sharing sheets with the service account robot email

# setup service account
sa = gspread.service_account(filename="sheets_credentials.json")

In [19]:
# open required sheets
#
# @ minimum column/s: {"YEAR", "RETAILER", "PROPERTY_TYPE", "TOTAL_DWELLING", "POSTCODE",
# "0L_DAY", "RATIO_0", "50L_DAY", "RATIO_50"}
sv_total_data_sh = sa.open("sv_total_data")
# @ minimum column/s: {"POSTCODE"}
holiday_postcodes_sh = sa.open("holiday_postcodes")
# @ minimum column/s: {"POSTCODE"}
exempt_postcodes_sh = sa.open("exempt_postcodes")
# @ minimum column/s = {"LOCALITY_NAME", "POSTCODE"}



# open sheet worksheets
sv_total_data_ws = sv_total_data_sh.get_worksheet(0)
holiday_postcodes_ws = holiday_postcodes_sh.get_worksheet(0)
exempt_postcodes_ws = exempt_postcodes_sh.get_worksheet(0)

# get data from worksheets
sv_total_data_wsd = sv_total_data_ws.get_all_records()
holiday_postcodes_wsd = holiday_postcodes_ws.get_all_records()
exempt_postcodes_wsd = exempt_postcodes_ws.get_all_records()

# convert worksheet to pd.DataFrame
sv_total_data_df = pd.DataFrame.from_dict(sv_total_data_wsd)
holiday_postcodes_df = pd.DataFrame.from_dict(holiday_postcodes_wsd)
exempt_postcodes_df = pd.DataFrame.from_dict(exempt_postcodes_wsd)

In [21]:
sv_total_data_df.head()

,YEAR,RETAILER,PROPERTY_TYPE,TOTAL_DWELLING,POSTCODE,0L_DAY,RATIO_0,50L_DAY,RATIO_50,EXEMPT_TAX_AREA,HOLIDAY
0,2020,GWW,residential,25825,3000,331,1.281704,3339,12.92933,tax,FALSE
1,2020,GWW,residential,2296,3002,11,0.479094,250,10.8885,tax,FALSE
2,2020,GWW,residential,4061,3003,35,0.861857,340,8.372322,tax,FALSE
3,2020,GWW,residential,6506,3008,75,1.152782,711,10.92837,tax,FALSE
4,2020,GWW,residential,10221,3011,190,1.858918,454,4.441835,tax,FALSE


In [22]:
holiday_postcodes_df.head()

,POSTCODE
0,3795
1,3797
2,3775
3,3777
4,3756


In [23]:
exempt_postcodes_df.head()

,POSTCODE
0,3020
1,3021
2,3022
3,3023
4,3024


In [24]:
# put holiday/exempt postcodes into sets
exempt_set = set(exempt_postcodes_df["POSTCODE"])
holiday_set = set(holiday_postcodes_df["POSTCODE"])

In [27]:
# re-do "EXEMPT_TAX" and "HOLIDAY" columns in total data
sv_total_data_df["EXEMPT_TAX_AREA"] = np.where(sv_total_data_df["POSTCODE"].isin(exempt_set), "exemption", "tax")
sv_total_data_df["HOLIDAY"] = np.where(sv_total_data_df["POSTCODE"].isin(holiday_set), True, False)

sv_total_data_df.head()

,YEAR,RETAILER,PROPERTY_TYPE,TOTAL_DWELLING,POSTCODE,0L_DAY,RATIO_0,50L_DAY,RATIO_50,EXEMPT_TAX_AREA,HOLIDAY
0,2020,GWW,residential,25825,3000,331,1.281704,3339,12.92933,tax,False
1,2020,GWW,residential,2296,3002,11,0.479094,250,10.8885,tax,False
2,2020,GWW,residential,4061,3003,35,0.861857,340,8.372322,tax,False
3,2020,GWW,residential,6506,3008,75,1.152782,711,10.92837,tax,False
4,2020,GWW,residential,10221,3011,190,1.858918,454,4.441835,tax,False
